In [1]:
import chromadb
import cv2
import easyocr
import fitz 
import numpy as np
import os
import pytesseract
import re
import requests
import shutil
import torch
import uuid

from pathlib import Path
from PIL import Image
from io import BytesIO
from PIL import Image, ImageEnhance, ImageFilter
from transformers import CLIPProcessor, CLIPModel
from PIL import Image, ImageEnhance
from chromadb.utils import embedding_functions
from pymongo import MongoClient
from sentence_transformers import SentenceTransformer



In [6]:

# Carregar o modelo CLIP
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def is_image_url(url: str) -> bool:
    try:
        response = requests.get(url, stream=True, timeout=5, allow_redirects=True)
        content_type = response.headers.get('Content-Type', '').lower()
        print(f"Content-Type recebido: {content_type}")  # Adicione esse print

        if any(image_type in content_type for image_type in ['image/jpeg', 'image/png', 'image/gif', 'image/jpg']):
            return True
        print(f"Erro: O conteúdo da URL não é uma imagem. Content-Type: {content_type}")
        return False
    except Exception as e:
        print(f"Erro ao verificar URL: {e}")
        return False

        
# Função para classificar a imagem
def classify_image(image_path: str) -> str:
    try:
        # Carregar a imagem
        if image_path.startswith('http'):
            response = requests.get(image_path)
            image = Image.open(BytesIO(response.content)).convert("RGB")
        else:
            image = Image.open(image_path).convert("RGB")

        # Definir possíveis categorias
        labels = ["Mapa", "Gráfico", "Diagrama", "Tabela", "Outro"]
        inputs = processor(text=labels, images=image, return_tensors="pt", padding=True)

        # Obter as similaridades
        outputs = model(**inputs)
        logits_per_image = outputs.logits_per_image
        probs = logits_per_image.softmax(dim=1)

        # Identificar a categoria com maior probabilidade
        max_index = probs.argmax().item()
        category = labels[max_index]
        confidence = probs[0][max_index].item()

        return f"Categoria: {category} (Confiança: {confidence:.2f})"

    except Exception as e:
        return f"Erro ao classificar a imagem: {e}"


# Função para extração estruturada de texto com EasyOCR
def extract_text_structure(image_path: str) -> dict:
    try:
        # Carregar a imagem
        if image_path.startswith('http'):
            response = requests.get(image_path)
            image = Image.open(BytesIO(response.content)).convert("RGB")
        else:
            image = Image.open(image_path).convert("RGB")

        # Pré-processamento
        image = image.convert("L")  # Escala de cinza
        image = ImageEnhance.Contrast(image).enhance(2)
        image = ImageEnhance.Sharpness(image).enhance(2)

        # Converter para numpy array para processamento com OpenCV
        image_np = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

        # Usar EasyOCR para detecção de texto
        reader = easyocr.Reader(['pt'])
        result = reader.readtext(image_np)

        # Organizando os textos extraídos
        titles, captions, texts = [], [], []
        for (bbox, text, prob) in result:
            if prob > 0.5:  # Filtro de confiança
                text_content = text.strip()

                # Classificação simples com base na posição do texto (exemplo)
                if bbox[1][1] < 50:  # Considerando a parte superior da imagem como título
                    titles.append(text_content)
                elif bbox[1][1] >= 50 and bbox[1][1] < 200:  # Considerando a parte intermediária como legenda
                    captions.append(text_content)
                else:  # O restante como texto normal
                    texts.append(text_content)

        return {
            "titles": titles,
            "captions": captions,
            "texts": texts
        }

    except Exception as e:
        return {"error": str(e)}


# Exemplo de uso
image_path = "https://smastr16.blob.core.windows.net/igeo/2012/03/mapa_aguas_subterraneas.jpg"
print(classify_image(image_path))
print(extract_text_structure(image_path))


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Categoria: Mapa (Confiança: 0.69)


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
/home/stoiccode/gitRepositories/Database-Chroma-RAG-Project/venv311/lib64/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'titles': ['MAPA DE ÁGUAS SUBTERRÂNEAS DO ESTADO DE SÃO PAULO'], 'captions': [], 'texts': ['#x']}


In [7]:
# Inicialização do modelo de embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# MongoDB
mongo_client = MongoClient("mongodb://localhost:27017/")
mongo_db = mongo_client["rag_db"]
images_collection = mongo_db["images"]

# ChromaDB
chroma_client = chromadb.PersistentClient(path="../chroma_db")
collection = chroma_client.get_or_create_collection(name="image_descriptions")


def store_image_info(image_url: str) -> None:
    try:
        # Verificar se o URL é uma imagem
        if not is_image_url(image_url):
            print(f"Erro: O conteúdo da URL não é uma imagem: {image_url}")
            return

        # Baixar a imagem
        response = requests.get(image_url, stream=True)
        response.raise_for_status()

        # Carregar a imagem
        image = Image.open(BytesIO(response.content)).convert("RGB")

        # Classificar a imagem
        category = classify_image(image_url)  # Aqui você pode integrar a função classify_image(image_url)
        extracted_info = extract_text_structure(image_url)

        # Gerar uma descrição concatenada para os embeddings
        description = f"{category}: " + " ".join(extracted_info["titles"] + extracted_info["texts"])
        embedding = embedding_model.encode(description).tolist()

        # Gerar UUID
        doc_id = str(uuid.uuid4())

        # Armazenar no MongoDB
        image_data = {
            "id": doc_id,
            "url": image_url,
            "category": category,
            "titles": extracted_info["titles"],
            "texts": extracted_info["texts"],
            "embedding": embedding
        }
        images_collection.insert_one(image_data)

        # Armazenar no ChromaDB
        collection.add(
            embeddings=[embedding],
            metadatas=[{
                "id": doc_id,
                "url": image_url,
            }],
            ids=[doc_id]
        )

        print(f"[OK] Informações da imagem armazenadas no MongoDB e ChromaDB com ID {doc_id}")

    except Exception as e:
        print(f"Erro ao armazenar a imagem: {e}")

# Exemplo de uso
image_links = [
    "https://smastr16.blob.core.windows.net/igeo/2012/03/mapa_aguas_subterraneas.jpg",
]

for link in image_links:
    store_image_info(link)



Content-Type recebido: image/jpeg


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


[OK] Informações da imagem armazenadas no MongoDB e ChromaDB com ID f3f28bf5-e657-4135-beb9-4165578e7ff7


In [5]:
#Teste de query

# Inicialização do modelo de embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Configurações do MongoDB
mongo_client = MongoClient("mongodb://localhost:27017/")
mongo_db = mongo_client["rag_db"]
images_collection = mongo_db["images"]

# Configurações do ChromaDB
chroma_client = chromadb.PersistentClient(path="../chroma_db")
collection = chroma_client.get_or_create_collection(name="image_descriptions")

# Modelo de Embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

def query_search(query: str, top_k: int = 5):
    try:
        # Gerar embedding da query
        query_embedding = embedding_model.encode(query).tolist()

        # Realizar a busca no ChromaDB
        results = collection.query(
            query_embeddings=[query_embedding],
            n_results=top_k
        )
        
        # Recuperar os IDs e distâncias encontrados
        found_ids = results["ids"][0]
        distances = results["distances"][0]

        # Exibir IDs e distâncias
        print(f"IDs encontrados: {found_ids}")
        print(f"Distanças: {distances}")

        # Recuperar os IDs encontrados
        found_ids = results["ids"][0]
        print(f"IDs encontrados: {found_ids}")

        # Buscar os documentos no MongoDB
        documents = images_collection.find({"id": {"$in": found_ids}})
        return list(documents)

    except Exception as e:
        print(f"Erro ao realizar a pesquisa: {e}")
        return []

# Exemplo de uso
if __name__ == "__main__":
    query = " Nossa equipe escreveu um trecho de documentação EIA relativo a replantio de vegetação nativa que será removida dos trechos da via, você poderia verificar se o conteúdo segue as normas ambientais e sugerir melhorias para o foco futuro da equipe?Durante os estudos feitos pela equipe, foi verificado a existência da arvore Vitex polygama Cham. em trechos da estrada. Depois de uma analise cuidadosa da equipe, deseja-se realizar seu replantio no parque natural de chico mendes em Sorocaba, e para cada representante da espécie retirada será plantado um novo dentro dos limites do parque citado."
    results = query_search(query)
    for doc in results:
        print(f"titulos: {doc['titles']} url: {doc['url']}")




IDs encontrados: ['f3f28bf5-e657-4135-beb9-4165578e7ff7']
Distanças: [1.0777101516723633]
IDs encontrados: ['f3f28bf5-e657-4135-beb9-4165578e7ff7']
titulos: ['MAPA DE ÁGUAS SUBTERRÂNEAS DO ESTADO DE SÃO PAULO'] url: https://smastr16.blob.core.windows.net/igeo/2012/03/mapa_aguas_subterraneas.jpg


In [23]:
chroma_client = chromadb.PersistentClient(path="./chroma_db")

In [3]:
# #limpar o banco de dados caso necessário
# # Conexão com o MongoDB
# from pymongo import MongoClient

# mongo_client = MongoClient("mongodb://localhost:27017")
# mongo_db = mongo_client["rag_db"]
# mongo_chunks = mongo_db["images"]

# # Remover todos os documentos da collection
# mongo_chunks.delete_many({})
# print("Coleção 'images' do MongoDB limpa com sucesso.")

Coleção 'images' do MongoDB limpa com sucesso.


In [5]:
# #limpar o chromadb caso necessário
# import chromadb

# # Inicializar o cliente ChromaDB
# chroma_client = chromadb.PersistentClient(path="../chroma_db")
# collection = chroma_client.get_or_create_collection(name="image_descriptions")

# # Buscar todos os IDs
# all_ids = collection.get()["ids"]

# # Remover todos os documentos pelo ID
# if all_ids:
#     collection.delete(ids=all_ids)
#     print(f"Removidos {len(all_ids)} documentos da coleção 'image_descriptions'.")
# else:
#     print("Nenhum documento encontrado para remoção no ChromaDB.")

Removidos 2 documentos da coleção 'image_descriptions'.
